In [1]:
# determine co-occurrence of interesting variants in genes from PCGC project with Stacia, Casey and Jeeves
import gzip
import pandas as pd
import re
import numpy as np
import seaborn as sns


### this is the only part of the script you should have to touch regularly ###

gene='MED13L'
outfile = '../results/med13l_pairs.tsv'
gen_vcf_file = 'vcfs/med13l.vcf.gz' # this should be the gzipped VCF for whichever gene from 1000Gs
trios_file = '../data/common+rare/cardio277_cases_freq30_list.txt' # this should be the cases (from Stacia)
ctrls_file = '../data/rare+rare/cardio277_controls_with_trios.txt' # this should be the controls file (from Stacia)
gene_list_file = '../data/rare+rare/cardio277_summary.txt' # this is simply used to get the gene list

In [2]:
# import vcf from 1000 Genomes
with gzip.open(gen_vcf_file, 'rb') as f: 
    gen_df = pd.read_table(f,sep='\t',comment='#',usecols=[1,*range(9,2513)],index_col=0,header=None)

# import trios data
trios_df_names = [1,2,3,4]
with open(trios_file,'r') as f:
    trios_df = pd.read_table(f, sep='\t', comment='#',names=trios_df_names, index_col=None)
    
# import gene list
with open(gene_list_file,'r') as f:
    gen_sum_df = pd.read_table(f, sep='\t', index_col=None, header=None)
gene_list = list(gen_sum_df[0][1:])

def get_vars_df(gen_df):
    var_pat = re.compile('1\|1|2\|2|0\|1|1\|0|2\|0|0\|2|1\|2|2\|1|0\|3|3\|0|1\|3|3\|1|2\|3|3\|2|3\|3')
    non_var_pat = re.compile('0\|0')

    # replace variants with 1 and non-variants with 0

    gen_df.replace(to_replace=[var_pat,non_var_pat],value=[1,0],inplace=True,regex=True)

    # make sure all values are int

    gen_df_int = gen_df.astype(int)
    
    return(gen_df_int)

def get_cooc_df(gen_df_int):

    # transpose dataframe to get cooccurrence matrix

    coocc = gen_df_int.dot(gen_df_int.T)

    # reset diagonal

    np.fill_diagonal(coocc.values, 0)

    # change index name to avoid later errors

    coocc = coocc.reindex(coocc.index.rename('Index'))
    
    return(coocc)

# plot clustermap (if desired)

# % matplotlib inline

# clusterplot = sns.clustermap(coocc)
# sns.heatmap(coocc)

# figure out which pairs of variants are present in cases for gene of interest

def get_pairs(gene, coocc):
    next_gene_index = gene_list.index(gene) + 1
    gene_index = trios_df[trios_df[1].str.contains(gene)].index.tolist()
    next_gene_index = trios_df[trios_df[1].str.contains(gene_list[next_gene_index])].index.tolist()

    rel_df = trios_df.loc[gene_index[0]:next_gene_index[0],:]

    # add new column to specify case when 2 SNPs for father or mother
    rel_df['check'] = rel_df[1].shift(1)
    rel_df['add'] = rel_df[2].shift(-1)
    rel_df['check2'] = rel_df[2].shift(2)

    father_snps = []
    mother_snps = []
    for index,row in rel_df.iterrows():
        if 'Father' in str(row[1]):
            father_snps.append(row[2]) 
            mother_snps.append(row['add'])
        elif len(str(row[1]))==2 and 'Father' in str(row['check']):
            father_snps.append(row[2])
            mother_snps.append(row['add'])
        elif len(str(row[1]))==2 and 'Mother' in str(row['check']):
            father_snps.append(row['check2'])
            mother_snps.append(row[2])

    cooc_freq_list = []
    cooc_list = []

    counter = -1
    for var in father_snps:
        counter += 1
        mvar = mother_snps[counter]
        if int(var) in coocc.index and int(mvar) in coocc.index:
            cooc = float(coocc.loc[int(var),int(mvar)])
        else:
            cooc = np.nan
        cooc_list.append(cooc)
        cooc_freq_list.append(cooc/2504.0)
    cooc_of_interest = pd.DataFrame({'Father SNP':father_snps,'Mother SNP':mother_snps,'Cooccurrence freq in 1000 Genomes':cooc_freq_list,'Cooc in 1000 Genomes':cooc_list})
    cooc_of_interest = cooc_of_interest.sort_values(by='Cooccurrence freq in 1000 Genomes', ascending=False)
    return(cooc_of_interest)

def save_cooc_results(gene, outfile):
    gen_df_int = get_vars_df(gen_df)
    coocc_df = get_cooc_df(gen_df_int)
    pairs_df = get_pairs(gene, coocc_df)
    pairs_df.to_csv(outfile,sep='\t')
    return(pairs_df)

In [ ]:
out_df = save_cooc_results(gene, outfile)

In [26]:
# check for these SNPs in controls

ctrl_names = [1,2,3,4]
with open(ctrls_file, 'r') as f:
    ctrl_df = pd.read_table(ctrls_file, names=ctrl_names, sep='\t')

In [37]:
# # how often do these SNPs occur individually in the controls?

# next_gene_index = gene_list.index(gene) + 1
# gene_index = ctrl_df[ctrl_df[1].str.contains(gene)].index.tolist()
# next_gene_index = ctrl_df[ctrl_df[1].str.contains(gene_list[next_gene_index])].index.tolist()

# rel_ctrls = ctrl_df.loc[gene_index[0]:next_gene_index[0],:]

In [4]:
print(out_df)

     Cooc in 1000 Genomes  Cooccurrence freq in 1000 Genomes Father SNP  \
52                   84.0                           0.033546   23859610   
16                   84.0                           0.033546   23859610   
82                   84.0                           0.033546   23859610   
74                   84.0                           0.033546   23859610   
70                   84.0                           0.033546   23859610   
64                   84.0                           0.033546   23859610   
62                   84.0                           0.033546   23874507   
46                   84.0                           0.033546   23859610   
44                   84.0                           0.033546   23859610   
39                   84.0                           0.033546   23874507   
36                   84.0                           0.033546   23874507   
33                   84.0                           0.033546   23859610   
30                   84.0